# 04 — Task 3: 12-Month Forecast (TSLA)

In [ ]:
# === Colab Setup ===
!pip install -U pip --quiet
!pip install "numpy<2.0" "scipy<1.11" --quiet
!pip install "pmdarima==2.0.3" --quiet
!pip install pandas numpy matplotlib seaborn scikit-learn statsmodels yfinance PyPortfolioOpt arch --quiet
import sys, os, platform
print("Python:", sys.version)
print("Platform:", platform.platform())
print("CWD:", os.getcwd())

In [ ]:
# === Configuration ===
START_DATE = "2015-07-01"
END_DATE   = "2025-07-31"
TICKERS    = ["TSLA", "BND", "SPY"]
BT_START   = "2024-08-01"
BT_END     = "2025-07-31"
TRADING_DAYS = 252
RISK_FREE_DAILY = 0.0

DATA_DIR = "data"
OUT_PLOTS = "outputs/plots"
OUT_MODELS = "outputs/models"
OUT_METRICS = "outputs/metrics"

for d in [DATA_DIR, OUT_PLOTS, OUT_MODELS, OUT_METRICS]:
    os.makedirs(d, exist_ok=True)


In [ ]:
# === Load ARIMA model and forecast ===
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from joblib import load
import pmdarima as pm

prices = pd.read_csv(f"{DATA_DIR}/prices_adj_close.csv", index_col=0, parse_dates=True)
tsla = prices['TSLA'].dropna()

# Load model if available; otherwise fit quickly
model_path = f"{OUT_MODELS}/tsla_arima.pkl"
if os.path.exists(model_path):
    arima_model = load(model_path)
else:
    arima_model = pm.auto_arima(tsla.loc[:'2023-12-31'], seasonal=False, stepwise=True,
                                error_action="ignore", suppress_warnings=True, max_p=5, max_q=5, max_d=2)

future_periods = 252  # ~12 months
fc_future, conf_future = arima_model.predict(n_periods=future_periods, return_conf_int=True)

last_date = prices.index.max()
future_index = pd.bdate_range(start=last_date + pd.tseries.offsets.BDay(1), periods=future_periods)
fc_series = pd.Series(fc_future, index=future_index, name="TSLA_Forecast")

plt.figure(figsize=(12,5))
plt.plot(prices['TSLA'].iloc[-500:], label="History")
plt.plot(fc_series, label="12M Forecast")
plt.fill_between(future_index, conf_future[:,0], conf_future[:,1], alpha=0.15, label="95% CI")
plt.title("TSLA 12-Month Forecast with 95% CI"); plt.legend(); plt.savefig(f"{OUT_PLOTS}/tsla_12m_forecast.png"); plt.show()

fc_series.to_csv(f"{OUT_METRICS}/tsla_forecast_12m.csv")
print("Saved forecast to", f"{OUT_METRICS}/tsla_forecast_12m.csv")


> Proceed to Notebook 05 to translate the forecast into expected returns and run the Efficient Frontier with PyPortfolioOpt.